In [3]:
import numpy as np
import random
qubit_0 =  np.array([1,0]).reshape(2,)
qubit_1 =  np.array([0,1]).reshape(2,)
qubit_000 = np.kron(np.kron(qubit_0,qubit_0),qubit_0)
qubit_111 = np.kron(np.kron(qubit_1,qubit_1),qubit_1)
c = random.random()
c




0.9712576547423558

In [4]:
qubit_final=(qubit_000 + c* qubit_111) * np.sqrt((1/(1+c**2)))
qubit_final

array([0.71734066, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.6967226 ])

In [8]:
import numpy as np
import random 
CNOT_matrix=np.array([[1,0,0,0],
                      [0,1,0,0],
                      [0,0,0,1],
                      [0,0,1,0]])
hadamard=(1/(np.sqrt(2)))*np.array([[1, 1],[1,-1]])

swap= np.array([[1,0,0,0],
                [0,0,1,0],
                [0,1,0,0],
                [0,0,0,1]])

qubit_0 =  np.array([1,0]).reshape(2,)
qubit_1 =  np.array([0,1]).reshape(2,)
qubit_00 = np.kron(qubit_0,qubit_0)
qubit_11 = np.kron(qubit_1,qubit_1)
c = random.random()
c=complex(2,4)
identity = np.identity(2)



In [7]:
#initializing the state
psi_complex = np.empty(4,dtype=complex)
psi_real = np.empty(4,dtype=float)

#taking the input of the state
for i in range(4):
    x=float(input("Element:")) #taking input for the matrix of the state
    
    #taking the input for various values of c

    choice = input("complex or real")
    if choice == "complex":
        c = complex(float(input("x")),float(input("y")))
        psi_complex[i]=c*x
        psi_1=(psi_complex.reshape(4,))/np.linalg.norm(psi_complex)


    elif choice == "real":
        c= float(input("Enter the real coefficent"))
        psi_real[i]=c*x
        psi_1=(psi_real.reshape(4,))/np.linalg.norm(psi_real)

        
    else:
        print("Invalid input")
        break


psi_2=psi_1

for i in range(int(input("no of iterations"))):

    psi = np.kron(psi_1,psi_2)
    

    #performing swap operation 
    swap_tensored =np.kron(identity,np.kron(swap,identity))
    swaped_state= np.matmul(swap_tensored,psi)


    #cnot operation on swapped qubits 
    cnot = np.kron(CNOT_matrix,CNOT_matrix)
    state_after_cnot = np.matmul(cnot,swaped_state)

    #swapping the states back 
    original_psi = np.matmul(swap_tensored,state_after_cnot)

    #hadamard operation 
    tensored_hadamard = np.kron(hadamard,np.kron(hadamard,np.identity(4)))
    final_state_after_hadamard =np.matmul(tensored_hadamard,original_psi)


    #measurement
    projector=np.array([[1,0],[0,0]])
    projector_tensored = np.kron(np.identity(4),np.kron(projector,projector))

    probablity = np.dot(np.conjugate(final_state_after_hadamard.reshape(1,16)),np.dot(projector_tensored,final_state_after_hadamard))


    #tracing out the measured copy
    first_copy_of_the_state = np.trace(final_state_after_hadamard.reshape(2,2,2,2),axis1=2,axis2=3)


    #creating bell state
    bell_state = (np.array([[1],[0],[0],[1]]) * (1/ np.sqrt(2))).reshape(4,)


    # preparation of density matrix of bell state and psi state by taking the outer product between the states
    density_matrix_bell = np.outer(np.conjugate(bell_state),bell_state) 

    #dividing by the trace of density matrix to make it 1
    density_matrix_pp = np.outer(np.conjugate(first_copy_of_the_state),first_copy_of_the_state) / np.trace(np.outer(np.conjugate(first_copy_of_the_state),first_copy_of_the_state))


    #calculation of fidelity
    x = np.matmul(np.conjugate(bell_state.reshape(1,4)),np.matmul(density_matrix_pp,bell_state))
    fidelity = x

    psi_1 = first_copy_of_the_state.reshape(4,)
    psi_2 = first_copy_of_the_state.reshape(4,)
    



final_state_after_hadamard , first_copy_of_the_state, probablity , fidelity, 






(array([ 6.40000000e+01, -3.15544362e-29, -3.15544362e-29,  6.40000000e+01,
        -3.47098798e-29, -3.50324616e-46, -3.50324616e-46, -3.47098798e-29,
        -3.47098798e-29, -3.50324616e-46, -3.50324616e-46, -3.47098798e-29,
         6.40000000e+01,  3.50324616e-46,  3.50324616e-46,  6.40000000e+01]),
 array([[ 1.28000000e+02, -6.94197597e-29],
        [-6.94197597e-29,  1.28000000e+02]]),
 array([8192.]),
 array([1.]))